<a href="https://colab.research.google.com/github/deanhoperobertson/Named-Enitty-Recognition/blob/master/BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving prepro.py to prepro.py


In [2]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 757kB 4.0MB/s 


In [0]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.model_selection import train_test_split

#cusotm packages
from prepro import readfile, readstring, partial_tags

#keras and tensorflow packages
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

#evaluation
from sklearn_crfsuite.metrics import flat_classification_report,flat_f1_score,flat_precision_score


In [0]:
#import data from my github repo
train_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/train.txt"
test_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/test.txt"
train = urllib.request.urlopen(train_url).read()
test = urllib.request.urlopen(test_url).read()
train = train.decode('utf-8')
test = test.decode('utf-8')

#preproces the txt file
train = readstring(train)
test = readstring(test)

#create corpus
corpus = train.copy()
corpus.extend(test)

In [27]:
words = []
tags = []
for sentence in corpus:
    for word in sentence:
        words.append(word[0])
        tags.append(word[1])
        
tags = partial_tags(tags)

setlabel=list(set(tags))
setlabel.remove('O')
        
words= [w.lower() for w in words]

n_words = len(set(words))
print("Number of words in the dataset: ", n_words)
n_tags = len(list(set(tags)))
print("Number of Labels: ", n_tags)

Number of words in the dataset:  24277
Number of Labels:  5


In [28]:
word2idx = {w: i + 2 for i, w in enumerate(list(set(words)))}
word2idx["UNK"] = 1 # Unknown words
word2idx["PAD"] = 0 # Padding
# Vocabulary Key:token_index -> Value:word
idx2word = {i: w for w, i in word2idx.items()}
print("The word 'rejects' is identified by the index: {}".format(word2idx["rejects"]))

The word 'rejects' is identified by the index: 11073


In [29]:
# The first entry is reserved for PAD
tag2idx = {t: i+1 for i, t in enumerate(list(set(tags)))}
tag2idx["PAD"] = 0
# Vocabulary Key:tag_index -> Value:Label/Tag
idx2tag = {i: w for w, i in tag2idx.items()}
print("The labels LOC (location) is identified by the index: {}".format(tag2idx["LOC"]))

The labels LOC (location) is identified by the index: 5


In [30]:
#Find the maxium length of the all the sentences in the corpus
length = []
for sentence in corpus:
  length.append(len(sentence))

MAX_LEN= max(length)
print("The maxium length of sentence is:",max(length))

The maxium length of sentence is: 124


In [31]:
# Convert each sentence from list of Token to list of word_index
X = [[word2idx[w[0].lower()] for w in s] for s in train]

# Padding each sentence to have the same lenght
X = pad_sequences(maxlen=MAX_LEN, sequences=X, padding="post", value=word2idx["PAD"])
X[0]

array([14319, 11073, 22134, 13356,   336, 10928,   946, 18134,   652,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0], dtype=int32)

In [32]:
def get_labeler(word):
  if word =="O":
    return word
  elif "MISC" in word:
    return word[-4:]
  else:
    return word[-3:]

# Convert Tag/Label to tag_index
y = [[tag2idx[get_labeler((w[1]))] for w in s] for s in train]

# Padding each sentence to have the same lenght
y = pad_sequences(maxlen=MAX_LEN, sequences=y, padding="post", value=tag2idx["PAD"])
y[0]

array([4, 1, 2, 1, 1, 1, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [0]:
# One-Hot encode categorical labels
y = [to_categorical(i, num_classes=n_tags+1) for i in y]

In [37]:
# Model definition
EMBEDDING=40

input = Input(shape=(MAX_LEN,))

model = Embedding(input_dim=len(word2idx)+1, output_dim=EMBEDDING, # n_words + 2 (PAD & UNK)
                  input_length=MAX_LEN, mask_zero=True)(input)

model = Bidirectional(LSTM(units=64, return_sequences=True,
                           recurrent_dropout=0.1))(model)


out = TimeDistributed(Dense(6, activation="softmax"))(model)
model = Model(input, out)

model.compile(optimizer="adam", loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 124)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 124, 40)           971200    
_________________________________________________________________
bidirectional_7 (Bidirection (None, 124, 128)          53760     
_________________________________________________________________
time_distributed_6 (TimeDist (None, 124, 6)            774       
Total params: 1,025,734
Trainable params: 1,025,734
Non-trainable params: 0
_________________________________________________________________


In [38]:
%%time
BATCH_SIZE = 200
EPOCHS=16


history = model.fit(X, np.array(y), batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.2, verbose=1)

Train on 11232 samples, validate on 2809 samples
Epoch 1/16
11232/11232 [==============================] - 59s 5ms/step - loss: 1.0748 - acc: 0.8082 - val_loss: 0.6385 - val_acc: 0.8504
Epoch 2/16
11232/11232 [==============================] - 54s 5ms/step - loss: 0.6293 - acc: 0.8248 - val_loss: 0.5177 - val_acc: 0.8510
Epoch 3/16
11232/11232 [==============================] - 54s 5ms/step - loss: 0.4674 - acc: 0.8318 - val_loss: 0.3826 - val_acc: 0.8625
Epoch 4/16
11232/11232 [==============================] - 54s 5ms/step - loss: 0.3299 - acc: 0.8759 - val_loss: 0.3298 - val_acc: 0.8865
Epoch 5/16
11232/11232 [==============================] - 54s 5ms/step - loss: 0.2641 - acc: 0.9098 - val_loss: 0.3017 - val_acc: 0.9023
Epoch 6/16
11232/11232 [==============================] - 54s 5ms/step - loss: 0.2067 - acc: 0.9383 - val_loss: 0.2659 - val_acc: 0.9238
Epoch 7/16
11232/11232 [==============================] - 54s 5ms/step - loss: 0.1416 - acc: 0.9633 - val_loss: 0.2359 - val_acc:

In [0]:
model.save("BiLSTM_model")

## Model Evaluation

#### Training Dataset

In [41]:
pred = model.predict(X, verbose=1)

14041/14041 [==============================] - 32s 2ms/step


In [42]:
# TRain Eval
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(y, -1)

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in preds]
y_tr_true_tag = [[idx2tag[i] for i in row] for row in y_tr_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag)
print(report)

score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted')
print(score)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

         LOC       0.17      0.95      0.29      8297
        MISC       0.28      0.89      0.42      4593
           O       0.10      0.99      0.19    169578
         ORG       0.32      0.89      0.48     10025
         PAD       0.00      0.00      0.00   1537463
         PER       0.26      0.94      0.41     11128

    accuracy                           0.11   1741084
   macro avg       0.19      0.78      0.30   1741084
weighted avg       0.02      0.11      0.03   1741084

0.0262934152911771


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [43]:
report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=setlabel)
print(report)
#F1 Score
score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted',labels=setlabel)
print(score)

              precision    recall  f1-score   support

        MISC       0.28      0.89      0.42      4593
         PER       0.26      0.94      0.41     11128
         ORG       0.32      0.89      0.48     10025
         LOC       0.17      0.95      0.29      8297

   micro avg       0.25      0.92      0.39     34043
   macro avg       0.26      0.92      0.40     34043
weighted avg       0.26      0.92      0.40     34043

0.40249172850993964


### Test Dataset

In [0]:
# Convert each sentence from list of Token to list of word_index
X_test = [[word2idx[w[0].lower()] for w in s] for s in test]

# Padding each sentence to have the same lenght
X_test = pad_sequences(maxlen=MAX_LEN, sequences=X_test, padding="post", value=word2idx["PAD"])
X_test

# Convert Tag/Label to tag_index
y_test = [[tag2idx[get_labeler((w[1]))] for w in s] for s in test]

# Padding each sentence to have the same lenght
y_test = pad_sequences(maxlen=MAX_LEN, sequences=y_test, padding="post", value=tag2idx["PAD"])

# One-Hot encode categorical labels
y_test = [to_categorical(i, num_classes=n_tags+1) for i in y_test]

In [46]:
pred = model.predict(X_test, verbose=1)

3453/3453 [==============================] - 8s 2ms/step


In [47]:
#pred_cat = model.predict(X_tr)
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(y_test, -1)

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in preds]
y_tr_true_tag = [[idx2tag[i] for i in row] for row in y_tr_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=setlabel)
print(report)

score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted',labels=setlabel)
print(score)

              precision    recall  f1-score   support

        MISC       0.12      0.67      0.20       918
         PER       0.19      0.57      0.28      2773
         ORG       0.13      0.63      0.21      2496
         LOC       0.04      0.81      0.07      1925

   micro avg       0.08      0.66      0.14      8112
   macro avg       0.12      0.67      0.19      8112
weighted avg       0.13      0.66      0.20      8112

0.20284940658031925
